In [ ]:
import sympy as sp

In [ ]:
def ctensor_tti(A,N,C,L,F,s,i,j,k,l):
    from sympy import KroneckerDelta as Delta 
    s = (A - 2 * N) * Delta(i,j) * Delta(k,l) + N * (Delta(i,k) * Delta(j,l) + Delta(i,l) * Delta(j,k))  + \
        (F-A+2*N) * (Delta(i,j) * s[k] * s[l] + Delta(k,l) * s[i] * s[j])  + \
        (L-N) * (Delta(i,k) * s[j] * s[l] + Delta(i,l) * s[j] * s[k] + Delta(j,k) * s[i] * s[l] + Delta(j,l) * s[i] * s[k])  + \
        (A + C -2 * F - 4 * L) * s[i] * s[j] * s[k] * s[l]
    
    return s

In [ ]:
rho,om = sp.symbols('rho,omega',real=True)
z = sp.symbols("z",real=True)
e3 = sp.Array([0,0,1])
k0 = sp.symbols('k',real=True)
phi = sp.symbols("phi",real=True)

# axis location
phi0 = sp.symbols("phi0",real=True)
theta0 = sp.symbols("theta0",real=True)
k_hat = sp.Array([1,0,0])
k = k_hat * k0
t_hat = sp.Array([0,1,0])

U,V,W = sp.symbols("U,V,W",real=False)
dU,dV,dW = sp.symbols("Udot,Vdot,Wdot",real=False)
I = sp.I
s = V * e3 + k_hat  * U  + t_hat  * W 
ds = dV * e3 + k_hat  * dU + t_hat  * dW 
s = s.as_mutable()
ds = ds.as_mutable()

# trial function
psi = sp.symbols("psi",real=True)
dpsi = sp.symbols("psidot",real=True)

dphi = sp.symbols("dphi",real=True)

# # tti tensor
axis_direc = sp.Array([sp.cos(dphi) * sp.sin(theta0),sp.sin(dphi) * sp.sin(theta0),sp.cos(theta0)])
#axis_direc = sp.Array([0,0,1])
#axis_direc = sp.Array([0,0,1])
A,N,C,L,F = sp.symbols('A,N,C,L,F',real=True)
c = sp.Array.zeros(3,3,3,3)
c = c.as_mutable()
for i in range(3):
    for j in range(3):
        for p in range(3):
            for q in range(3):
                c[i,j,p,q] = ctensor_tti(A,N,C,L,F,axis_direc,i,j,p,q)

T = sp.Array([0,0,0])
T = T.as_mutable()
G = T * 1
Lag = 0
for i in range(3):
    a = 0
    b = 0
    Lag = Lag + rho * om**2 * s[i] * sp.conjugate(s[i]) 
    for j in range(3):
        for p in range(3):
            for q in range(3):
                a = a + e3[j] * c[i,j,p,q] * (-I * k[q] * s[p] + e3[q] * ds[p])
                b = b + I * k[j] * c[i,j,p,q] * (-I * k[q] * s[p] + e3[q] * ds[p])

                Lag = Lag - (I * k[j] * sp.conjugate(s[i]) + e3[j] * sp.conjugate(ds[i])) * c[i,j,p,q] *  \
                      (-I * k[q] * s[p] + e3[q] * ds[p])
    T[i] = a
    G[i] = b

out = T * dpsi  + G * psi

In [ ]:
def replace_fortran_str(text):
    #temp:str = text.replace("+ &\n      "," + ")
    temp:str = text.replace(" &\n      ","")
    temp = temp.replace("+"," + ")
    temp = temp.replace("-"," - ")
    temp = temp.replace("  -"," -")
    temp = temp.replace("  +"," +")
    temp = temp.replace("-  ","-  ")
    temp = temp.replace("+  ","+ ")
    #temp:str = temp.replace("- &\n      "," - ")
    

    # convert to list
    a = list(temp)

    # find all operators
    idx = [i for i in range(len(temp)) if temp.startswith(' + ', i)]
    idx_m = [i for i in range(len(temp)) if temp.startswith(' - ', i)]
    idx.extend(idx_m)
    idx.sort()
    idx.append(len(a) - 1)
    idx1 = [False for i in range(len(idx))]
    
    # add change line symbol at +/-
    for i in range(80,len(a),72):
        # loc = 
        # loc = -1
        # for j in range(len(idx) - 1):
        #     if i >= idx[j] and i < idx[j+1]:
        #         loc = j 
        #         break
        # if loc != -1 and idx1[loc] == False :
        #     s = a[idx[loc]]
        #     if s == ' ':
        #         a[idx[loc]] = ' &\n      '
        #     else:
        #         a[idx[loc]] = s + ' &\n      '
        #     idx1[loc] = True
        # find closest
        loc = -1; dist = 999999
        for j in range(len(idx) - 1):
            id = abs(i - idx[j])
            if dist > id:
                loc = j
                dist = id
        if loc != -1 and idx1[loc] == False :
            s = a[idx[loc]]
            if s == ' ':
                a[idx[loc]] = ' &\n      '
            else:
                a[idx[loc]] = s + ' &\n    '
            idx1[loc] = True
        
    # replace the last &\n with \n
    if ' &\n      ' in a[-1]:
        a[-1] = a[-1].replace(' &\n      ','')
    
    # now replace sin/cos function with corresponding arrays
    f_temp =  ''.join(a)
    f_temp = f_temp.replace("cos(theta0)","costh0")
    f_temp = f_temp.replace("sin(theta0)","sinth0")
    f_temp = f_temp.replace("cos(dphi)","cosphi")
    f_temp = f_temp.replace("sin(dphi)","sinphi")
    f_temp = f_temp.replace("+  ","+ ")
    f_temp = f_temp.replace("-  ","- ")
    f_temp = f_temp.replace("cmplx(0,1)","imag_i")


    return f_temp

In [ ]:
funclist = {
    "re":'real',
    "im": "aimag",
}


# open a txt file to write all functions
fileid = open("../src//tti_subs.f90","w")
fileid.write("!===========================================================================\n")
fileid.write("!============================= AUTO CODE FROM SYMPY ========================\n")
fileid.write("!===========================================================================\n")
fileid.write("\n")

# loop each comp of out
for i in range(3):

    fileid.write(f"subroutine get_comp{i + 1} (NGL,A,C,F,L,N,theta0,dphi,jaco, &\n")
    fileid.write(f"                        weight,hp,hpT, K0U,K0V,K0W,K1U,K1V,K1W,&\n")
    fileid.write(f"                        K2U,K2V,K2W) bind(c,name='get_comp{i+1}_')\n")
    fileid.write("  use iso_c_binding,only: c_int,dp => c_double ,dcp => c_double_complex\n")
    fileid.write(f"  implicit none\n\n")
    fileid.write(f"  integer(c_int),value,intent(in)         :: NGL\n")
    fileid.write(f"  real(dp),dimension(NGL),intent(in)  :: A,C,F,L,N,theta0,dphi,weight\n")
    fileid.write(f"  real(dp),dimension(NGL,NGL),intent(in)  :: hp,hpT\n")
    fileid.write(f"  real(dp),value :: jaco \n")
    fileid.write(f"  complex(dcp),dimension(NGL,NGL),intent(out) :: K1U,K1V,K1W,K2U,K2V,K2W,K0U,K0V,K0W \n")
    fileid.write(f"  \n  !local vars\n")
    fileid.write(f"  integer(c_int)        :: i,j \n")
    fileid.write(f"  complex(dcp),dimension(NGL)  :: temp\n")
    fileid.write(f"  real(dp),dimension(NGL) :: costh0, sinth0,cosphi,sinphi\n")
    fileid.write(f"  complex(dcp),parameter :: imag_i = cmplx(0.0,1.0,kind=dcp)\n")
    fileid.write(f"\n  !init matrices\n")
    for p in range(3):
        fileid.write(f"  K{p}U(:,:) = (0.0,0.0); K{p}V(:,:) = (0.0,0.0); K{p}W(:,:) = (0.0,0.0);\n")
    fileid.write(f"  costh0 = cos(theta0); sinth0 = sin(theta0);\n")
    fileid.write(f"  cosphi = cos(dphi); sinphi = sin(dphi);\n")
    fileid.write(f"\n")

    # as polynomial of k and loop k^0,k^1,k^2
    expr = out[i].as_poly(k0)
    for p in range(3): # 
        k_power_expr = expr.coeff_monomial(k0**p)
        
        # as polynomial of field U,V,W,dV,dU,dW
        field_expr_poly = k_power_expr.as_poly(U,V,W,dV,dU,dW)
        f = [[U,dU],
             [W,dW],
             [V,dV]]
        psi_lst = [psi,dpsi]
        for ii in range(3):
            for jj in range(2):
                # weak form
                weak_poly = field_expr_poly.coeff_monomial(f[ii][jj]).as_poly(psi,dpsi)
                for kk in range(2):
                    weak_expr = weak_poly.coeff_monomial(psi_lst[kk])
                    if weak_expr == 0:
                        continue
                    print(f"k^{p}, {f[ii][jj]} {psi_lst[kk]} = {weak_expr}")
                    #weak_expr.replace("cos(theta0)","costh0")
                    f_out = sp.fcode(weak_expr,standard=2008,source_format='free',user_functions=funclist,assign_to="temp(:)")
                    #display(f_out)
                    f_out = replace_fortran_str(f_out)
                    
                    # cache temporary arrays
                    fileid.write(f"  ! k^{p}, {f[ii][jj]} {psi_lst[kk]}\n")
                    fileid.write(f"  {f_out}\n")

                    # compute weak form
                    cur_arr = f'K{p}{f[ii][0]}'
                    if jj == 0 and kk == 0: # U psi
                        fileid.write(f'  do i=1,NGL; {cur_arr}(i,i) = {cur_arr}(i,i) + temp(i) * weight(i) * jaco; enddo\n')
                    elif jj == 0 and kk == 1: # U dpsi
                        fileid.write("  do j=1,NGL; do i=1,NGL; \n")
                        fileid.write(f"    {cur_arr}(i,j) = {cur_arr}(i,j) + temp(j) * weight(j) * hpT(i,j)\n")
                        fileid.write("  enddo; enddo; \n")
                        pass
                    elif jj == 1 and kk == 0: # dU psi
                        fileid.write("  do j=1,NGL; do i=1,NGL; \n")
                        fileid.write(f"    {cur_arr}(i,j) = {cur_arr}(i,j) + temp(i) * weight(i) * hp(i,j)\n")
                        fileid.write("  enddo; enddo; \n")
                        pass 
                    else:
                        fileid.write("  do j=1,NGL; do i=1,NGL; \n")
                        fileid.write(f"    {cur_arr}(i,j) = {cur_arr}(i,j) + sum(weight * temp / jaco * hp(:,j) * hp(:,i))\n")
                        fileid.write("  enddo; enddo; \n")
                    
                    #fileid.write(f"  {out_str}")
                    fileid.write("\n")
    # transpose 
    fileid.write("  !transpose\n")
    for p in range(3):
        fileid.write(f"  K{p}U = transpose(K{p}U); K{p}V = transpose(K{p}V); K{p}W = transpose(K{p}W);\n")

    # end
    fileid.write(f"\nend subroutine get_comp{i + 1}\n\n")


# kernels
fileid.write(f"subroutine get_kernels (NGL,k,A,C,F,L,N,theta0,dphi, &\n")
fileid.write(f"                        U,V,W,Udot,Vdot,Wdot,Kwvnm,&\n")
fileid.write(f"                        KA,KC,KF,KL,KN,Ktheta0,Kdphi,dL_dkv) bind(c,name='get_kernels_')\n")
fileid.write("  use iso_c_binding,only: c_int,dp => c_double ,dcp => c_double_complex\n")
fileid.write(f"  implicit none\n\n")
fileid.write(f"  integer(c_int),value,intent(in)         :: NGL\n")
fileid.write(f"  real(dp),value,intent(in)               :: k\n")
fileid.write(f"  real(dp),dimension(NGL),intent(in)      :: A,C,F,L,N,theta0,dphi\n")
fileid.write(f"  complex(dcp),dimension(NGL),intent(in)  :: U,V,W,Udot,Vdot,Wdot\n")
fileid.write(f"  real(dcp),dimension(NGL),intent(out)    :: Kwvnm,KA,KC,KF,KL,KN,Ktheta0,Kdphi\n")
fileid.write(f"  real(dcp),dimension(NGL,2),intent(out)  :: dL_dkv\n")
fileid.write(f"  \n  !local vars\n")
fileid.write(f"  complex(dcp),dimension(NGL)  :: temp\n")
fileid.write(f"  real(dp),dimension(NGL) :: costh0, sinth0,cosphi,sinphi\n")
fileid.write(f"  complex(dcp),parameter :: imag_i = cmplx(0.0,1.0,kind=dcp)")
fileid.write(f"\n\n  !init matrices\n")
fileid.write(f"  costh0 = cos(theta0); sinth0 = sin(theta0);\n")
fileid.write(f"  cosphi = cos(dphi); sinphi = sin(dphi);\n")
fileid.write("\n")

# compute kernels d{Lag}/d{Param}
for param in [A,C,F,L,N,theta0,dphi,k0]:
    expr = sp.diff(Lag,param)
    param_out = param 
    if param == k0:
        param_out = 'wvnm'
    f_out = sp.fcode(expr,standard=2008,source_format='free',user_functions=funclist,assign_to=f"temp(:)")
    f_out = replace_fortran_str(f_out)
    fileid.write(f"  {f_out}\n")
    fileid.write(f"  K{param_out}(:) = real(temp(:),kind=dp)\n\n")

# compute group velocity derivative
print(k,c.shape,len(s))
for m in range(2):
    dLag_dkvec = 0
    for i in range(3):
        for j in range(3):
            for p in range(3):
                dLag_dkvec += -I * sp.conjugate(s[i]) * c[i,m,p,j] * (-I * k[j] * s[p] + e3[j] * ds[p])   \
                      - (I * k[j] * sp.conjugate(s[i]) + e3[j] * sp.conjugate(ds[i])) * c[i,j,p,m] * (-I * s[p]) 
    f_out = sp.fcode(dLag_dkvec,standard=2008,source_format='free',user_functions=funclist,assign_to=f"temp(:)")
    f_out = replace_fortran_str(f_out)
    fileid.write(f"  {f_out}\n")
    fileid.write(f"  dL_dkv(:,{m+1}) = real(temp(:),kind=dp)\n")

fileid.write(f"\nend subroutine get_kernels\n\n")



fileid.close()

import numpy as np

def ctensor_tti(A,N,C,L,F,s,i,j,k,l):
    Delta = lambda i,j: (i==j) * 1.
    from sympy import KroneckerDelta as Delta 
    s = (A - 2 * N) * Delta(i,j) * Delta(k,l) + N * (Delta(i,k) * Delta(j,l) + Delta(i,l) * Delta(j,k))  + \
        (F-A+2*N) * (Delta(i,j) * s[k] * s[l] + Delta(k,l) * s[i] * s[j])  + \
        (L-N) * (Delta(i,k) * s[j] * s[l] + Delta(i,l) * s[j] * s[k] + Delta(j,k) * s[i] * s[l] + Delta(j,l) * s[i] * s[k])  + \
        (A + C -2 * F - 4 * L) * s[i] * s[j] * s[k] * s[l]
    
    return s

In [ ]:
sp.diff(Lag,om)

# define symbols
np.random.seed(10)
rho,om = 2.5,np.pi * 2 / 12
e3 = np.array([0,0,1])
k0 = om / 3.5
phi = 20 * np.pi / 180

# axis location
phi0 = 130 * np.pi / 180
theta0 = np.pi / 4
k = np.array([1,0,0]) * k0
t = np.array([0,1,0])
k1 = np.array([np.cos(phi),np.sin(phi),0]) * k0
t1 = np.array([-np.sin(phi),np.cos(phi),0])

U,V,W = np.random.randn(3) + 1j * np.random.randn(3)
dU,dV,dW = np.random.randn(3) + 1j * np.random.randn(3)
I = 0. + 1.0j
s = V * e3 + I * k / k0 * U - I * t  * W 
ds = dV * e3 + I * k / k0 * dU - I * t  * dW 
s1 = V * e3 + I * k1 / k0 * U - I * t1  * W 
ds1 = dV * e3 + I * k1 / k0 * dU - I * t1  * dW 

# trial function
psi = 1.5
dpsi = 2.01231

# # tti tensor
axis_direc = np.array([np.cos(phi0-phi) * np.sin(theta0),np.sin(phi0-phi) * np.sin(theta0),np.cos(theta0)])
axis_direc1 = np.array([np.cos(phi0) * np.sin(theta0),np.sin(phi0) * np.sin(theta0),np.cos(theta0)])
A,N,C,L,F = rho * 3.5**2 + rho * 3.5**2 * 0.5 * np.random.rand((5)) 
c = np.zeros((3,3,3,3))
c1 = c * 1
for i in range(3):
    for j in range(3):
        for p in range(3):
            for q in range(3):
                c[i,j,p,q] = ctensor_tti(A,N,C,L,F,axis_direc,i,j,p,q)
                c1[i,j,p,q] = ctensor_tti(A,N,C,L,F,axis_direc1,i,j,p,q)

T = np.zeros((3),dtype=np.complex128)
G = T * 0
T1 = T * 0; G1 = G * 0
Lag = 0
Lag1 = 0
for i in range(3):
    a = 0.
    b = 0.
    a1 = 0.
    b1 = 0.
    Lag = Lag + rho * om**2 * s[i] * np.conjugate(s[i]) 
    Lag1 = Lag1 + rho * om**2 * s1[i] * np.conjugate(s1[i]) 
    for j in range(3):
        for p in range(3):
            for q in range(3):
                a = a + e3[j] * c[i,j,p,q] * (-I * k[q] * s[p] + e3[q] * ds[p])
                b = b + I * k[j] * c[i,j,p,q] * (-I * k[q] * s[p] + e3[q] * ds[p])

                a1 += e3[j] * c1[i,j,p,q] * (-I * k1[q] * s1[p] + e3[q] * ds1[p])
                b1 += I * k1[j] * c1[i,j,p,q] * (-I * k1[q] * s1[p] + e3[q] * ds1[p])

                Lag = Lag - (I * k[j] * np.conjugate(s[i]) + e3[j] * np.conjugate(ds[i])) * c[i,j,p,q] *  \
                      (-I * k[q] * s[p] + e3[q] * ds[p])
                Lag1 = Lag1 - (I * k1[j] * np.conjugate(s1[i]) + e3[j] * np.conjugate(ds1[i])) * c1[i,j,p,q] *  \
                      (-I * k1[q] * s1[p] + e3[q] * ds1[p])
    T[i] = a
    G[i] = b
    T1[i] = a1 
    G1[i] = b1

out = -T * dpsi  - G * psi + rho * om**2 * s * psi

display(Lag-Lag1)
display(T[0] - T1.dot(k1/k0))
display(T[1] - T1.dot(t1))
display(T[2] - T1[2])